In [ ]:
import torch
#from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
#from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data = pd.read_csv('/content/formal_informal.csv').head(10000)
data.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   flags        10000 non-null  object
 1   instruction  10000 non-null  object
 2   category     10000 non-null  object
 3   intent       10000 non-null  object
 4   response     10000 non-null  object
dtypes: object(5)
memory usage: 390.8+ KB


In [ ]:
data.drop(['flags','category','response'], axis = 1, inplace = True)

In [ ]:
x = data['instruction'].map(lambda instruction: re.sub('[^a-zA-Z\s]+', ' ', instruction).lower())
y = data['intent']
y
#text = data['content']


0         cancel_order
1         cancel_order
2         cancel_order
3         cancel_order
4         cancel_order
             ...      
9995    create_account
9996    create_account
9997    create_account
9998    create_account
9999    create_account
Name: intent, Length: 10000, dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Encode labels as integers
label_encoder = LabelEncoder()
label = label_encoder.fit_transform(data['intent'])


In [ ]:
# Split the data
#x = data['text']
train_text, val_text, train_label, val_label = train_test_split(x, label, test_size=0.2, random_state=42)

In [29]:
#!pip install -U accelerate
#!pip install wandb
#!pip install -U transformers accelerate

'''

!pip install -U transformers accelerate
!pip install cartopy
!apt-get -qq install -y graphviz && pip install pydot
!apt-get -qq install -y libfluidsynth1
!pip install torch
'''

# Import necessary libraries
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score, log_loss, matthews_corrcoef, hamming_loss, jaccard_score
from torch.utils.data import Dataset
import wandb
import io

# Initialize wandb (optional)
wandb.init(project="roberta-classification")

# Define the RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(set(label)))

# Tokenize the data
train_encodings = tokenizer(list(train_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)
val_encodings = tokenizer(list(val_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Convert label to long
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_label)
val_dataset = SentimentDataset(val_encodings, val_label)

training_args = TrainingArguments(
    output_dir='./roberta-output',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    logging_dir='./logs_roberta',
    logging_steps=100,
    evaluation_strategy="epoch",
    report_to="wandb"
)

# Define the compute_metrics function
def compute_metrics(eval_results):
    if hasattr(eval_results, 'predictions') and hasattr(eval_results, 'label_ids'):
        logits = eval_results.predictions
        labels = eval_results.label_ids
        preds = logits.argmax(-1)

        # Accuracy
        accuracy = accuracy_score(labels, preds)

        # Precision, Recall, F1 Score
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

        # Confusion Matrix
        conf_matrix = confusion_matrix(labels, preds)

        # AUC-ROC (for binary classification)
        if len(set(labels)) == 2:
            auc_roc = roc_auc_score(labels, preds)
        else:
            auc_roc = None

        # Log Loss
        logloss = log_loss(labels, logits, labels=list(set(labels)))

        # MCC
        mcc = matthews_corrcoef(labels, preds)

        # Hamming Loss
        h_loss = hamming_loss(labels, preds)

        # Jaccard Score
        jaccard = jaccard_score(labels, preds, average='weighted')

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "confusion_matrix": conf_matrix,
            "auc_roc": auc_roc,
            "log_loss": logloss,
            "mcc": mcc,
            "hamming_loss": h_loss,
            "jaccard_score": jaccard
        }
    else:
        return {"error": "eval_results does not have the required attributes"}

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
metrics = compute_metrics(eval_results)
print(f"Metrics: {metrics}")

eval/accuracy,▁▁
eval/f1,▁▁
eval/hamming_loss,▁▁
eval/jaccard_score,▁▁
eval/log_loss,▁▁
eval/loss,▁▁
eval/mcc,▁▁
eval/precision,▁▁
eval/recall,▁▁
eval/runtime,▁█
eval/samples_per_second,█▁


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Confusion Matrix,Auc Roc,Log Loss,Mcc,Hamming Loss,Jaccard Score,Runtime,Samples Per Second,Steps Per Second
1,0.004300,0.006977,0.999000,0.999010,0.999000,0.999000,[[206 2 0 0 0 0 0 0 0 0 0] [ 0 202 0 0 0 0 0 0 0 0 0] [ 0 0 187 0 0 0 0 0 0 0 0] [ 0 0 0 193 0 0 0 0 0 0 0] [ 0 0 0 0 203 0 0 0 0 0 0] [ 0 0 0 0 0 190 0 0 0 0 0] [ 0 0 0 0 0 0 205 0 0 0 0] [ 0 0 0 0 0 0 0 206 0 0 0] [ 0 0 0 0 0 0 0 0 214 0 0] [ 0 0 0 0 0 0 0 0 0 171 0] [ 0 0 0 0 0 0 0 0 0 0 21]],None,0.033180,0.998892,0.001000,0.998010,12.426700,160.944000,5.070000


Evaluation results: {'eval_loss': 0.006977061275392771, 'eval_accuracy': 0.999, 'eval_precision': 0.9990098039215686, 'eval_recall': 0.999, 'eval_f1': 0.99900004759525, 'eval_confusion_matrix': array([[206,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 202,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 187,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 193,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 203,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 190,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 205,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 206,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 214,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 171,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  21]]), 'eval_auc_roc': None, 'eval_log_loss': 0.03318008841240308, 'eval_mcc': 0.9988919925691584, 'eval_ha

In [ ]:
'''
# Save the model
buffer = io.BytesIO()
torch.save(model.state_dict(), buffer)
buffer.seek(0)
model.load_state_dict(torch.load(buffer))
model.save_pretrained()
tokenizer.save_pretrained()
'''

'\n# Save the model\nbuffer = io.BytesIO()\ntorch.save(model.state_dict(), buffer)\nbuffer.seek(0)\nmodel.load_state_dict(torch.load(buffer))\nmodel.save_pretrained()\ntokenizer.save_pretrained()\n'

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


TypeError: '<' not supported between instances of 'str' and 'int'

In [28]:
# Import necessary libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score, log_loss, matthews_corrcoef, hamming_loss, jaccard_score
from torch.utils.data import Dataset
import wandb
import io

# Initialize wandb (optional)
wandb.init(project="bert-classification")

# Define the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(label)))

# Tokenize the data
train_encodings = tokenizer(list(train_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)
val_encodings = tokenizer(list(val_text), truncation=True, padding='max_length', return_tensors='pt', max_length=128)

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # Convert label to long
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_label)
val_dataset = SentimentDataset(val_encodings, val_label)

training_args = TrainingArguments(
    output_dir='./bert-output',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    logging_dir='./logs_bert',
    logging_steps=100,
    evaluation_strategy="epoch",
    report_to="wandb"
)

# Define the compute_metrics function
def compute_metrics(eval_results):
    if hasattr(eval_results, 'predictions') and hasattr(eval_results, 'label_ids'):
        logits = eval_results.predictions
        labels = eval_results.label_ids
        preds = logits.argmax(-1)

        # Accuracy
        accuracy = accuracy_score(labels, preds)

        # Precision, Recall, F1 Score
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

        # Confusion Matrix
        conf_matrix = confusion_matrix(labels, preds)

        # AUC-ROC (for binary classification)
        if len(set(labels)) == 2:
            auc_roc = roc_auc_score(labels, preds)
        else:
            auc_roc = None

        # Log Loss
        logloss = log_loss(labels, logits, labels=list(set(labels)))

        # MCC
        mcc = matthews_corrcoef(labels, preds)

        # Hamming Loss
        h_loss = hamming_loss(labels, preds)

        # Jaccard Score
        jaccard = jaccard_score(labels, preds, average='weighted')

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "confusion_matrix": conf_matrix,
            "auc_roc": auc_roc,
            "log_loss": logloss,
            "mcc": mcc,
            "hamming_loss": h_loss,
            "jaccard_score": jaccard
        }
    else:
        return {"error": "eval_results does not have the required attributes"}

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")
metrics = compute_metrics(eval_results)
print(f"Metrics: {metrics}")

eval/loss,▁▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▂▃▃▄▅▆▆▇████
train/global_step,▁▂▃▃▄▅▆▆▇████
train/grad_norm,█▂▁▁▁▁▁▁▁▁
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▁▁▁▁▁▁▁▁▁
eval/error,eval_results is not ...
eval/loss,0.00174


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Confusion Matrix,Auc Roc,Log Loss,Mcc,Hamming Loss,Jaccard Score,Runtime,Samples Per Second,Steps Per Second
1,0.001900,0.001743,1.000000,1.000000,1.000000,1.000000,[[208 0 0 0 0 0 0 0 0 0 0] [ 0 202 0 0 0 0 0 0 0 0 0] [ 0 0 187 0 0 0 0 0 0 0 0] [ 0 0 0 193 0 0 0 0 0 0 0] [ 0 0 0 0 203 0 0 0 0 0 0] [ 0 0 0 0 0 190 0 0 0 0 0] [ 0 0 0 0 0 0 205 0 0 0 0] [ 0 0 0 0 0 0 0 206 0 0 0] [ 0 0 0 0 0 0 0 0 214 0 0] [ 0 0 0 0 0 0 0 0 0 171 0] [ 0 0 0 0 0 0 0 0 0 0 21]],None,0.013793,1.000000,0.000000,1.000000,13.158300,151.996000,4.788000


Evaluation results: {'eval_loss': 0.0017429263098165393, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_confusion_matrix': array([[208,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 202,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 187,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 193,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 203,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 190,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 205,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 206,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 214,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0, 171,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  21]]), 'eval_auc_roc': None, 'eval_log_loss': 0.013792505134975035, 'eval_mcc': 1.0, 'eval_hamming_loss': 0.0, 'eval_jaccard_score': 1.0, 

In [ ]:
# Save the model
buffer = io.BytesIO()
torch.save(model.state_dict(), buffer)
buffer.seek(0)
model.load_state_dict(torch.load(buffer))
model.save_pretrained()
tokenizer.save_pretrained()